In [19]:
# import the required libraries
import numpy as np
import pandas as pd

In [2]:
df1=pd.read_csv("Dataset/Aurbindo Marg  15 min 1-20 may.csv")

In [3]:
df1.shape

(1825, 4)

In [4]:
df2=pd.read_csv("Dataset/Sirifort 15 min 1-20 may.csv")

In [5]:
df2.shape

(1825, 4)

In [6]:
df3=pd.read_csv("Dataset/Patparganj 15min 1-20 may.csv")

In [7]:
df3.shape

(1825, 4)

In [8]:
df4=pd.read_csv("Dataset/abu fazal 15 min 1-20 may.csv")

In [9]:
df4.shape

(1825, 4)

In [10]:
df5=pd.read_csv("Dataset/Aurbindo(Robin) 15 min 1-20 may.csv")

In [11]:
df5.shape

(1825, 4)

#### Data Preprocessing

In [12]:
df3.isnull().sum()

CreatedDate                    0
pm25_Piyush: Living room     348
pm10_Piyush: Living room     348
aqi_Piyush: Living room      348
dtype: int64

In [13]:
df3.tail()

,CreatedDate,pm25_Piyush: Living room,pm10_Piyush: Living room,aqi_Piyush: Living room
1820,19-05-2024 23:15,37.0,83.0,83.0
1821,19-05-2024 23:30,36.0,77.0,77.0
1822,19-05-2024 23:45,35.0,75.0,75.0
1823,20-05-2024 00:00,35.0,74.0,74.0
1824,20-05-2024 00:15,34.0,71.0,71.0


In [14]:
#drop the null values
df3.dropna()

,CreatedDate,pm25_Piyush: Living room,pm10_Piyush: Living room,aqi_Piyush: Living room
0,01-05-2024 00:15,32.0,85.0,85.0
1,01-05-2024 00:30,32.0,85.0,85.0
2,01-05-2024 00:45,32.0,84.0,84.0
3,01-05-2024 01:00,31.0,83.0,83.0
4,01-05-2024 01:15,34.0,91.0,91.0
...,...,...,...,...
1820,19-05-2024 23:15,37.0,83.0,83.0
1821,19-05-2024 23:30,36.0,77.0,77.0
1822,19-05-2024 23:45,35.0,75.0,75.0
1823,20-05-2024 00:00,35.0,74.0,74.0


#### Creating a new dataframe to store the avg of pollutant and filtering the date

In [15]:
df1['CreatedDate'] = pd.to_datetime(df1['CreatedDate'], format='%d-%m-%Y %H:%M')

# Filter data for each day
start_date = pd.Timestamp('2024-05-01')
end_date = pd.Timestamp('2024-05-20')

# Creating an empty DataFrame to store the hourly average data
hourly_avg_df1 = pd.DataFrame(columns=['Date', 'Hour', 'AQI', 'Pm2.5', 'Pm10'])

#### Hourly avg Calculation and then concatenation and creating pivot table

In [16]:
for day in pd.date_range(start=start_date, end=end_date, freq='D'):
    df1_day = df1[df1['CreatedDate'].dt.date == day.date()]

    # Group by hour and calculate the mean for AQI, Pm2.5, and Pm10 for each hour can add other variable if required
    hourly_avg_aqi = df1_day.groupby(df1_day['CreatedDate'].dt.hour)['aqi_Surender: Living room'].mean()
    hourly_avg_pm25 = df1_day.groupby(df1_day['CreatedDate'].dt.hour)['pm25_Surender: Living room'].mean()
    hourly_avg_pm10 = df1_day.groupby(df1_day['CreatedDate'].dt.hour)['pm10_Surender: Living room'].mean()

    # Create a DataFrame for the hourly average data
    hourly_avg_data = pd.DataFrame({
        'Hour': hourly_avg_aqi.index,
        'AQI': hourly_avg_aqi.values,
        'Pm2.5': hourly_avg_pm25.values,
        'Pm10': hourly_avg_pm10.values
    })
    hourly_avg_data['Date'] = day.date()

    # Concatenate the current day's hourly average data to the main DataFrame
    hourly_avg_df1 = pd.concat([hourly_avg_df1, hourly_avg_data], ignore_index=True)


# "Pivot_table" the DataFrame to create a pivot table
pivot_table1 = pd.pivot_table(hourly_avg_df1, index='Hour', columns='Date', values=['AQI', 'Pm2.5', 'Pm10'])
daily_avg = pivot_table1.mean(axis=0)

# Appending the "Daily Avg" row to the pivot table
pivot_table1.loc['Daily Avg'] = daily_avg

# Convert the 'Daily Avg' row to numeric values
pivot_table1.loc['Daily Avg'] = pivot_table1.loc['Daily Avg'].astype(float)

print(pivot_table1)

                  AQI                                                \
Date       2024-05-01  2024-05-02 2024-05-03  2024-05-04 2024-05-05   
Hour                                                                  
0          135.333333  108.750000  154.00000  222.500000  232.25000   
1          127.000000  130.750000  211.00000  209.750000  221.50000   
2          197.250000  135.000000  312.50000  223.250000  222.00000   
3          182.750000  134.750000  358.50000  231.750000  195.75000   
4          106.750000  146.750000  357.50000  228.000000  162.00000   
5           84.500000  143.250000  341.75000  314.000000  216.25000   
6           95.500000  216.000000  329.25000  360.750000  317.25000   
7          114.000000  195.000000  334.00000  372.500000  275.00000   
8          119.500000  194.250000  289.75000  379.000000  213.25000   
9          222.250000  245.000000  295.00000  368.250000  352.00000   
10          98.500000  288.500000  137.25000  343.250000  246.25000   
11    

C:\Users\talha\AppData\Local\Temp\ipykernel_15124\3372701117.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  hourly_avg_df1 = pd.concat([hourly_avg_df1, hourly_avg_data], ignore_index=True)


In [18]:
pivot_table1

AQI                                                \
Date       2024-05-01  2024-05-02 2024-05-03  2024-05-04 2024-05-05   
Hour                                                                  
0          135.333333  108.750000  154.00000  222.500000  232.25000   
1          127.000000  130.750000  211.00000  209.750000  221.50000   
2          197.250000  135.000000  312.50000  223.250000  222.00000   
3          182.750000  134.750000  358.50000  231.750000  195.75000   
4          106.750000  146.750000  357.50000  228.000000  162.00000   
5           84.500000  143.250000  341.75000  314.000000  216.25000   
6           95.500000  216.000000  329.25000  360.750000  317.25000   
7          114.000000  195.000000  334.00000  372.500000  275.00000   
8          119.500000  194.250000  289.75000  379.000000  213.25000   
9          222.250000  245.000000  295.00000  368.250000  352.00000   
10          98.500000  288.500000  137.25000  343.250000  246.25000   
11          60.000000   87.750000  121.50000  252.750000  258.75000   
12          54.750000   77.500000  115.00000  243.500000  168.00000   
13          59.750000   85.500000   93.25000  169.000000  155.00000   
14          61.000000   76.000000  120.75000  131.750000  147.00000   
15          72.250000   73.000000  127.00000  165.750000  123.25000   
16          63.500000   68.750000  126.00000  117.000000  132.25000   
17          60.750000   77.000000  174.75000  143.000000  178.25000   
18          90.500000  107.500000  180.00000  248.500000  160.00000   
19          73.250000  121.250000  186.25000  240.250000  155.25000   
20         125.000000  132.250000  220.75000  302.750000  188.00000   
21         104.500000  132.500000  221.00000  313.500000  206.50000   
22          87.250000  133.750000  263.75000  247.750000  219.25000   
23          91.000000  140.750000  279.25000  227.500000  225.25000   
Daily Avg  103.618056  135.479167  222.90625  252.333333  207.09375   

                                                                     ...  \
Date      2024-05-06  2024-05-07  2024-05-08 2024-05-09  2024-05-10  ...   
Hour                                                                 ...   
0          250.25000  290.250000  230.750000   182.5000  214.000000  ...   
1          222.00000  316.250000  211.250000   183.5000  214.250000  ...   
2          181.50000  325.250000  224.500000   152.5000  198.000000  ...   
3          198.00000  322.500000  214.500000   139.0000  194.750000  ...   
4          199.25000  323.500000  209.250000   130.0000  203.750000  ...   
5          210.00000  345.250000  199.250000   134.7500  213.500000  ...   
6          264.50000  338.750000  195.000000   143.7500  217.500000  ...   
7          315.00000  363.750000  199.000000   138.0000  213.000000  ...   
8          294.25000  441.500000  187.250000   179.2500  213.250000  ...   
9          222.25000  447.500000  337.500000   170.7500  229.250000  ...   
10         192.75000  429.250000  211.750000   160.7500  202.750000  ...   
11         123.75000  212.500000  199.500000   169.2500  206.750000  ...   
12         144.00000  197.750000  201.500000   164.0000  212.000000  ...   
13         120.25000  176.250000  195.750000   164.5000  198.500000  ...   
14         113.75000  176.500000  184.500000   171.0000  205.250000  ...   
15         107.00000  177.000000  175.500000   175.7500  204.750000  ...   
16         113.25000  199.750000  164.250000   162.5000  203.000000  ...   
17         135.75000  188.000000  171.750000   186.5000  214.500000  ...   
18         108.25000  187.000000  181.250000   160.7500  207.750000  ...   
19         166.25000  212.750000  185.500000   185.0000  206.500000  ...   
20         214.75000  226.000000  205.250000   209.7500  212.000000  ...   
21         211.75000  221.750000  208.500000   194.0000  177.750000  ...   
22         224.50000  229.750000  197.250000   200.0000  233.250000  ...   
23         245.75000  229.250000  190.000000   208.75

In [17]:
# change the file name that you want to save
new_file_path_Aurobindo_Marg = "New_Aurobindo.csv"
pivot_table1.to_csv(new_file_path_Aurobindo_Marg, header=True)
#chnage the file name here too
print("New_Aurobindo_Marg hOURLY average AQI saved to CSV file:", new_file_path_Aurobindo_Marg)

New_Aurobindo_Marg hOURLY average AQI saved to CSV file: New_Aurobindo_Marg_hourly_avg_aqi.csv
